# **Tugas Besar Wawasan Global TIK**

> **Analisis Sentimen Kebijakan Pemerintah Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier Pada Sosial Media Twitter**

<br>

**Oleh Kelompok 152 :**
1. Fadhlurrahman Akbar Nasution (1301194258) / IF-42-GAB01
2. Firdaus Putra Kurniyanto (1301190385) / IF-42-GAB01
3. Ignasius Nindra Karisma Forestyanto  (1301194138)  / IF-42-GAB02
4. Kurniadi Ahmad Wijaya (1301194024) / IF-42-GAB01
5. Priyan Fadhil Supriyadi (1301190442) / IF-42-GAB01

<br>

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap kebijakan pemerintah mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

<br>
